In [2]:
import syft as sy
from syft.core.node.new.queue_stash import QueueItem

In [26]:
domain = sy.Worker.named('Domain', processes=1)

SQLite Store Path:
!open file:///tmp/79fa33618d8eaa1fc55f6f8a2d34c65e.sqlite

> Starting Worker: Domain - 79fa33618d8eaa1fc55f6f8a2d34c65e - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]


In [27]:
client = domain.root_client

In [28]:
from syft.core.node.new.api import SyftAPICall

In [29]:
call = SyftAPICall(
    node_uid=domain.id, path="data_subject.get_all", args=(), blocking=False, kwargs={}
)

In [30]:
signed_api_call = call.sign(client.credentials)

In [31]:
signed_result = domain.handle_api_call(signed_api_call)

In [39]:
signed_result.message.data

```python
class QueueItem:
  id: str = 06fa65db835441c693701308c494d90d
  node_uid: str = 79fa33618d8eaa1fc55f6f8a2d34c65e
  result: str = None
  resolved: str = False

```

In [42]:
signed_result.message.data.resolve

<class 'syft.core.node.new.response.SyftNotReady'>: 06fa65db835441c693701308c494d90d not ready yet.

In [44]:
import zmq
from zmq.devices import monitored_queue
from zmq.backend
ctx = zmq.Context()

ins = ctx.socket(zmq.ROUTER)
outs = ctx.socket(zmq.DEALER)
mons = ctx.socket(zmq.PUB)

In [48]:
ins, outs, mons

(<zmq.Socket(zmq.ROUTER) at 0x7fef416fa580>,
 <zmq.Socket(zmq.DEALER) at 0x7fef416fa9a0>,
 <zmq.Socket(zmq.PUB) at 0x7fef416faa00>)

In [51]:
# configure_sockets(ins,outs,mons)
monitored_queue(ins, outs, mons, in_prefix=b'in', out_prefix=b'out')

KeyboardInterrupt: 

In [54]:
def test_default_mq_args():
    device = dev = zmq.devices.ThreadMonitoredQueue(zmq.ROUTER, zmq.DEALER, zmq.PUB)
    dev.setsockopt_in(zmq.LINGER, 0)
    dev.setsockopt_out(zmq.LINGER, 0)
    dev.setsockopt_mon(zmq.LINGER, 0)
    # this will raise if default args are wrong
    dev.start()
    return device

In [55]:
device = test_default_mq_args()

In [24]:
import zmq
from zmq import devices
import time

class MessageQueue:
    def __init__(self):
        self.sockets = []
        self.context = zmq.Context()
        
    def create_bound_pair(
        self, type1=zmq.PAIR, type2=zmq.PAIR, interface='tcp://127.0.0.1'
    ):
        """Create a bound socket pair using a random port."""
        s1 = self.context.socket(type1)
        s1.setsockopt(zmq.LINGER, 0)
        port = s1.bind_to_random_port(interface)
        s2 = self.context.socket(type2)
        s2.setsockopt(zmq.LINGER, 0)
        s2.connect(f'{interface}:{port}')
        self.sockets.extend([s1, s2])
        return s1, s2
    
    def build_queue(self):
        pub, sub = self.create_bound_pair(zmq.PUB, zmq.SUB)
        return pub, sub
        
    def build_device(self, mon_sub=b"", in_prefix=b'in', out_prefix=b'out'):
            self.device = devices.ThreadMonitoredQueue(zmq.PUB, zmq.SUB, zmq.PUB,
                                                in_prefix, out_prefix)
            alice = self.context.socket(zmq.PUB)
            bob = self.context.socket(zmq.SUB)
            mon = self.context.socket(zmq.SUB)

            aport = alice.bind_to_random_port('tcp://127.0.0.1')
            bport = bob.bind_to_random_port('tcp://127.0.0.1')
            mport = mon.bind_to_random_port('tcp://127.0.0.1')
            mon.setsockopt(zmq.SUBSCRIBE, mon_sub)

            self.device.connect_in("tcp://127.0.0.1:%i"%aport)
            self.device.connect_out("tcp://127.0.0.1:%i"%bport)
            self.device.connect_mon("tcp://127.0.0.1:%i"%mport)
            self.device.start()
            time.sleep(.2)
            try:
                # this is currenlty necessary to ensure no dropped monitor messages
                # see LIBZMQ-248 for more info
                mon.recv_multipart(zmq.NOBLOCK)
            except zmq.ZMQError:
                pass
            self.sockets.extend([alice, mon])
            return alice, bob, mon
        
    def teardown_device(self):
        for socket in self.sockets:
            socket.close()
            del socket
        del self.device

In [25]:
queue = MessageQueue()

In [23]:

# queue.teardown_device()

In [26]:
pub, sub, mon = queue.build_device()

In [27]:
pub.send(b'asdasd')

In [ ]:
sub.recv()

In [22]:
s.recv()

KeyboardInterrupt: 

In [105]:
zmq.PAIR??